In [1]:
import random
import string
import time
import tensorflow as tf
import numpy as np
from scipy import sparse

In [284]:
list_of_strs = [''.join([string.ascii_lowercase[random.randrange(0, 26)] for i in range(random.randrange(4, 20))])
                for j in range(500)]

In [285]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [286]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [287]:
char_dict[' '] = max(char_dict.values())+1

In [288]:
max_length = max([len(i) for i in list_of_strs])

In [289]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix([0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [290]:
n_tiles = 500 # dense_strs.shape[0]

In [291]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()

In [292]:
max_length = dense_strs.shape[1]

In [293]:
dense_strs = sparse_strs.toarray()

In [294]:
dense_strs.shape

(500, 513)

In [295]:
vector_data = tf.placeholder(dtype=tf.float32, shape=dense_strs.shape)

In [296]:
tiled_vector_data = tf.reshape(tf.tile(vector_data, multiples=[1, n_tiles]), 
                               shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [297]:
tile_ref_data = tf.reshape(tf.tile(vector_data, multiples=[n_tiles, 1]), 
                           shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [298]:
not_equal = tf.cast(tf.equal(tile_ref_data, tiled_vector_data), dtype=tf.float32)

In [299]:
dist_sum = tf.reduce_sum(not_equal, axis=-1)

In [300]:
nearest_neighbors = tf.nn.top_k(dist_sum, k=2, sorted=True)

In [301]:
_, hamming_top_args = tf.nn.top_k(nearest_neighbors.values[:,1], k=nearest_neighbors.values.shape[0])

In [302]:
#nearest_neighbors.values[hamming_top_args]

In [303]:
with tf.Session() as sess:
    t = sess.run([hamming_top_args, nearest_neighbors.indices, nearest_neighbors.values], {vector_data:dense_strs})

In [309]:
def map_nearest_neighbor(string_list, top_args, top_values):
    
    if type(string_list)!=np.array:
        string_list = np.array(string_list)
    
    results = []
    values = []
    for idx, top_arg in enumerate(top_args):
        results.append(string_list[top_arg])
        values.append(top_values[top_args])
    return np.array(results), np.array(values)

In [311]:
t[1][t[0]]

array([[ 21, 456],
       [ 49,  81],
       [ 81,  49],
       [456,  21],
       [  7, 320],
       [ 34,  81],
       [ 42, 143],
       [ 63, 200],
       [ 92,  34],
       [121, 129],
       [129, 121],
       [132, 147],
       [143,  42],
       [147,  21],
       [150, 378],
       [174, 428],
       [176, 426],
       [187, 340],
       [200,  42],
       [212,  63],
       [233, 354],
       [237, 375],
       [241, 378],
       [263, 200],
       [315,  21],
       [317, 351],
       [320,   7],
       [333,  21],
       [340, 187],
       [351, 317],
       [354, 233],
       [357,  34],
       [375, 237],
       [378, 150],
       [426,  63],
       [428, 174],
       [451,  63],
       [453, 493],
       [458, 469],
       [467,  34],
       [469,  42],
       [477,  63],
       [493,  81],
       [ 16,  34],
       [ 59, 106],
       [ 94, 241],
       [103,  34],
       [106,  34],
       [119, 176],
       [179,  92],
       [213, 119],
       [227, 143],
       [231,

In [312]:
t[2][t[0]]

array([[513., 509.],
       [513., 509.],
       [513., 509.],
       [513., 509.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 507.],
       [513., 505.],
       [513., 505.],
       [513., 505.],
       [513., 505.],
       [513.,

In [310]:
nn_results = map_nearest_neighbor(list_of_strs, t[1][t[0]], t[2][t[0]])

IndexError: index 21 is out of bounds for axis 0 with size 2

In [306]:
nn_results[0][0:2]

array([['eleh', 'mlea'],
       ['fvmd', 'nemd']], dtype='<U19')

In [308]:
nn_results[1]

array([[[[513., 507.],
         [513., 483.]],

        [[513., 505.],
         [513., 503.]],

        [[513., 503.],
         [513., 505.]],

        ...,

        [[513., 485.],
         [513., 503.]],

        [[513., 483.],
         [513., 507.]],

        [[513., 483.],
         [513., 503.]]],


       [[[513., 507.],
         [513., 483.]],

        [[513., 505.],
         [513., 503.]],

        [[513., 503.],
         [513., 505.]],

        ...,

        [[513., 485.],
         [513., 503.]],

        [[513., 483.],
         [513., 507.]],

        [[513., 483.],
         [513., 503.]]],


       [[[513., 507.],
         [513., 483.]],

        [[513., 505.],
         [513., 503.]],

        [[513., 503.],
         [513., 505.]],

        ...,

        [[513., 485.],
         [513., 503.]],

        [[513., 483.],
         [513., 507.]],

        [[513., 483.],
         [513., 503.]]],


       ...,


       [[[513., 507.],
         [513., 483.]],

        [[513., 505.],
   